In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd 
data_train = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Kannada/kannada_sentiment_full_train.tsv', sep = '\t')
data_val = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Kannada/kannada_sentiment_full_dev.tsv', sep = '\t')
data_test = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Kannada/kannada_sentiment_full_test_withoutlabels.tsv', sep = '\t')

In [14]:
from sklearn import preprocessing
import numpy as np

le = preprocessing.LabelEncoder()

le.fit(data_train['category'])
print(le.classes_)

y_train_index = le.transform(data_train['category'])
y_val_index = le.transform(data_val['category'])


['Mixed feelings' 'Negative' 'Positive' 'not-Kannada' 'unknown state']


In [15]:
from keras.utils.np_utils import to_categorical
y_train_cal  =  to_categorical(y_train_index)
y_val_cal  =  to_categorical(y_val_index)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer(ngram_range=(1,6), analyzer = 'char_wb', max_features = 50000)
#tfidfvec = TfidfVectorizer(ngram_range=(1,3), max_features = 30000)
tfidf_train = tfidfvec.fit_transform(data_train['text'])
tfidf_val = tfidfvec.transform(data_val['text'])

In [17]:
x_train = tfidf_train
x_test = tfidf_val
y_train = y_train_cal
y_test =y_val_cal

In [18]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(4096, input_dim = tfidf_train.shape[1], activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 4096)              204804096 
_________________________________________________________________
dropout_4 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 325       
Total params: 206,934,917
Trainable params: 206,934,917
Non-trainable params: 0
________________________________________

In [19]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(x_train.toarray(), y_train, validation_data=(x_test.toarray(), y_test), epochs=20, batch_size=20, verbose=2 )

Epoch 1/20
311/311 - 14s - loss: 0.3624 - accuracy: 0.5705 - val_loss: 0.3068 - val_accuracy: 0.6512
Epoch 2/20
311/311 - 13s - loss: 0.2222 - accuracy: 0.7661 - val_loss: 0.3190 - val_accuracy: 0.6614
Epoch 3/20
311/311 - 13s - loss: 0.1124 - accuracy: 0.8902 - val_loss: 0.4564 - val_accuracy: 0.6512
Epoch 4/20
311/311 - 13s - loss: 0.0698 - accuracy: 0.9314 - val_loss: 0.6091 - val_accuracy: 0.6310
Epoch 5/20
311/311 - 13s - loss: 0.0472 - accuracy: 0.9486 - val_loss: 0.5857 - val_accuracy: 0.6295
Epoch 6/20
311/311 - 13s - loss: 0.0369 - accuracy: 0.9585 - val_loss: 0.6784 - val_accuracy: 0.6382
Epoch 7/20
311/311 - 13s - loss: 0.0317 - accuracy: 0.9652 - val_loss: 0.7179 - val_accuracy: 0.6454
Epoch 8/20
311/311 - 13s - loss: 0.0266 - accuracy: 0.9691 - val_loss: 0.8234 - val_accuracy: 0.6281
Epoch 9/20
311/311 - 13s - loss: 0.0249 - accuracy: 0.9722 - val_loss: 0.9032 - val_accuracy: 0.6324
Epoch 10/20
311/311 - 13s - loss: 0.0247 - accuracy: 0.9739 - val_loss: 0.8506 - val_accura

In [21]:
tfidf_test = tfidfvec.transform(data_test['text'])

In [22]:
predictions_test_submission = model.predict(tfidf_test.toarray())

In [25]:
predictions_test_submission

array([[6.27814301e-10, 3.96231981e-09, 9.99997377e-01, 6.85449706e-08,
        2.47665776e-06],
       [8.34990176e-04, 6.85240608e-04, 7.79213727e-01, 1.19174086e-01,
        1.00092001e-01],
       [1.33145898e-06, 3.01309311e-09, 4.04716593e-05, 4.29477723e-06,
        9.99953866e-01],
       ...,
       [5.43304985e-16, 8.00601827e-13, 1.00000000e+00, 3.94089032e-11,
        5.44785788e-12],
       [9.99830365e-01, 5.28267208e-07, 2.01034745e-05, 3.82885560e-07,
        1.48651539e-04],
       [7.12246845e-07, 2.51298671e-09, 9.99998212e-01, 1.85177498e-07,
        8.76227261e-07]], dtype=float32)

In [26]:
import numpy as np
predictions_test_submission = np.asarray(predictions_test_submission)

In [27]:
import numpy as np
predictions_test_submission1 = np.zeros_like(predictions_test_submission)
predictions_test_submission1[np.arange(len(predictions_test_submission)), predictions_test_submission.argmax(1)] = 1

In [28]:
predictions_test_submission1

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [29]:
predictions_test_submission1_final = np.argmax(predictions_test_submission1, axis=1)

In [30]:
predictions_test_submission1_final1 = le.inverse_transform(predictions_test_submission1_final)

In [31]:
predictions_test_submission1_final1

array(['Positive', 'Positive', 'unknown state', 'not-Kannada', 'Positive',
       'unknown state', 'Positive', 'not-Kannada', 'Positive', 'Positive',
       'Negative', 'Positive', 'not-Kannada', 'Mixed feelings',
       'Positive', 'Positive', 'Positive', 'Positive', 'Negative',
       'Negative', 'Positive', 'Positive', 'Positive', 'Positive',
       'not-Kannada', 'unknown state', 'not-Kannada', 'Positive',
       'Mixed feelings', 'Negative', 'Mixed feelings', 'not-Kannada',
       'not-Kannada', 'unknown state', 'Negative', 'unknown state',
       'unknown state', 'Positive', 'Positive', 'not-Kannada',
       'Mixed feelings', 'unknown state', 'Positive', 'Positive',
       'unknown state', 'not-Kannada', 'unknown state', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'unknown state',
       'unknown state', 'unknown state', 'Positive', 'Positive',
       'Positive', 'Positive', 'not-Kannada', 'Positive', 'Positive',
       'Mixed feelings', 'Positive', 'Mixed 

In [32]:
data_test['category'] = predictions_test_submission1_final1

In [33]:
data_test

,id,text,category
0,Kan_1,ಈ ಹಾಡನ್ನು ಹಾಡಿದ ವಿಜಯ ಪ್ರಕಾಶ voice ಯಾರಿಗೆಲ್ಲಾ ಇ...,Positive
1,Kan_2,Jai D Boss,Positive
2,Kan_3,Signature move,unknown state
3,Kan_4,Super song bro,not-Kannada
4,Kan_5,Wow Super agi helidira sir,Positive
...,...,...,...
763,Kan_764,Thu thukali trailer,Negative
764,Kan_765,Siri gannadam galge haakbitallapa Thu yaro ni...,Negative
765,Kan_766,ನಾವೇನಾದರು ಸ್ವಾಬಿಮಾನಿಯಾಗಿ ಬದುಕನ್ನು ಕಟ್ಟಿಕೊಳ್ಳಲು...,Positive
766,Kan_767,ದಿಯಾ ನೀವು ಸೋಷಿಯಲ್ ಮೀಡಿಯಾ ಅಲ್ಲಿ ಕಾಣ್ತಿಲ್ಲ.nದಯವಿ...,Mixed feelings


In [34]:
data_test.to_csv("/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Kannada/DNN_Kanada_64.tsv", sep="\t", index=False)

In [35]:
check_submission = pd.read_csv("/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Kannada/DNN_Kanada_64.tsv", sep="\t")

In [36]:
check_submission

,id,text,category
0,Kan_1,ಈ ಹಾಡನ್ನು ಹಾಡಿದ ವಿಜಯ ಪ್ರಕಾಶ voice ಯಾರಿಗೆಲ್ಲಾ ಇ...,Positive
1,Kan_2,Jai D Boss,Positive
2,Kan_3,Signature move,unknown state
3,Kan_4,Super song bro,not-Kannada
4,Kan_5,Wow Super agi helidira sir,Positive
...,...,...,...
763,Kan_764,Thu thukali trailer,Negative
764,Kan_765,Siri gannadam galge haakbitallapa Thu yaro ni...,Negative
765,Kan_766,ನಾವೇನಾದರು ಸ್ವಾಬಿಮಾನಿಯಾಗಿ ಬದುಕನ್ನು ಕಟ್ಟಿಕೊಳ್ಳಲು...,Positive
766,Kan_767,ದಿಯಾ ನೀವು ಸೋಷಿಯಲ್ ಮೀಡಿಯಾ ಅಲ್ಲಿ ಕಾಣ್ತಿಲ್ಲ.nದಯವಿ...,Mixed feelings
